In [13]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
from scipy.io import loadmat
np.random.seed(1)
data = loadmat('/content/drive/MyDrive/Colab Notebooks/ADA/SML/digit.mat')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
def generate_train_data(data, i, j):
    x = np.concatenate([data['X'][:, :, i], data['X'][:, :, j]], axis=1)
    x = np.transpose(x, (1, 0))
    y = np.concatenate([np.ones(500), -np.ones(500)])
    return x, y


def generate_test_data(data):
    x = data['T'][:, :, :]
    return x


def build_design_mat(x1, x2, bandwidth=1.):
    return np.exp(
        -np.sum((x1[:, None] - x2[None]) ** 2, axis=-1) / (2 * bandwidth ** 2))


def optimize_param(design_mat, y, regularizer):
    return np.linalg.solve(
        design_mat.T.dot(design_mat) + regularizer * np.identity(len(y)),
        design_mat.T.dot(y))


def predict(train_data, test_data, theta):
    return build_design_mat(train_data, test_data).T.dot(theta)

In [15]:
#training 1分半くらいかかる
theta_arr = np.empty((10, 10, 1000))
for i in range(9):
    for j in range(i+1, 10):
        x, y = generate_train_data(data, i, j)
        design_mat = build_design_mat(x, x)
        theta_arr[i, j] = optimize_param(design_mat, y, 1)

In [16]:
#メモリ節約用、1回のみ実行、２クラス比較のtraindataを格納
train_data_arr = np.empty((10, 10, 1000, 256))
for i in range(9):
    for j in range(10):
        train_data_arr[i,j] = np.transpose(np.concatenate([data['X'][:, :, i], data['X'][:, :, j]], axis=1), (1,0))

In [17]:
#test 1分半くらいかかる
confusion_matrix = np.zeros((10, 10), dtype=np.int64)

for label in range(10):
    test_data = np.transpose(data['T'][:, :, label], (1, 0))
    print("label=",label)
    for tx in test_data:
        num_arr = np.zeros((10,), dtype=np.int64)
        for i in range(9):
            for j in range(i+1, 10):
                prediction = predict(train_data_arr[i,j], tx, theta_arr[i,j])
                num_arr[i] += np.sum(np.where(prediction > 0, 1, 0))
                num_arr[j] += np.sum(np.where(prediction < 0, 1, 0))
        pred_label = np.argmax(num_arr)
        confusion_matrix[label, pred_label] += 1

print(confusion_matrix)
print("accuracy =", np.trace(confusion_matrix)/2000)

label= 0
label= 1
label= 2
label= 3
label= 4
label= 5
label= 6
label= 7
label= 8
label= 9
[[200   0   0   0   0   0   0   0   0   0]
 [  0 193   1   0   0   0   2   3   1   0]
 [  0   0 192   0   3   0   2   2   1   0]
 [  1   0   0 189   1   3   0   0   6   0]
 [  0   3   4   0 187   0   1   1   2   2]
 [  0   2   0   0   2 195   0   0   0   1]
 [  0   0   0   3   0   0 192   2   3   0]
 [  0   1   4   0   3   0   0 187   2   3]
 [  0   0   0   2   0   0   1   0 197   0]
 [  0   1   1   0   0   0   0   0   0 198]]
accuracy = 0.965
